<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_%C2%AB%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2_%D0%90%D0%9E%D0%A2%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Сделать классификацию данных fakenews**

Используя ноутбук занятия и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [1]:
# Сначала выполним базовую настройку и загрузку данных
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузка данных
df = pd.read_csv('Constraint_Train.csv')
print(f"Размер датасета: {df.shape}")
print(df['label'].value_counts())

Размер датасета: (6420, 3)
label
real    3360
fake    3060
Name: count, dtype: int64


##SKLEARN

In [9]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Функция для улучшенной предобработки текста
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление URL-адресов
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Удаление упоминаний и хэштегов
    text = re.sub(r'@\w+|#\w+', '', text)

    # Удаление цифр
    text = re.sub(r'\d+', '', text)

    # Удаление специальных символов, кроме букв и пробелов
    text = re.sub(r'[^\w\s]', '', text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [11]:
# Лемматизация
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [12]:
# Применяем предобработку
df['cleaned_tweet'] = df['tweet'].apply(preprocess_text)
df['cleaned_tweet'] = df['cleaned_tweet'].apply(lemmatize_text)

In [13]:
# Удаляем стоп-слова
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df['cleaned_tweet'] = df['cleaned_tweet'].apply(remove_stopwords)

In [14]:
# Векторизация с TF-IDF и n-граммами
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),  # Используем унарные и биграммы
    min_df=5,
    max_df=0.7,
    stop_words='english'
)

X = vectorizer.fit_transform(df['cleaned_tweet'])
y = df['label'].apply(lambda x: 1 if x == 'real' else 0)

In [15]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### Способ 1.1: Логистическая регрессия с настройкой

In [16]:
# Способ 1.1: Логистическая регрессия с настройкой
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'max_iter': [1000],
    'solver': ['lbfgs', 'liblinear']
}

log_reg = LogisticRegression(random_state=42)
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры для логистической регрессии:")
print(grid_search.best_params_)

y_pred = grid_search.predict(X_test)
print("\nРезультаты логистической регрессии:")
print(classification_report(y_test, y_pred))
print(f"F1-score (macro): {f1_score(y_test, y_pred, average='macro'):.4f}")

Лучшие параметры для логистической регрессии:
{'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}

Результаты логистической регрессии:
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       918
           1       0.92      0.92      0.92      1008

    accuracy                           0.91      1926
   macro avg       0.91      0.91      0.91      1926
weighted avg       0.91      0.91      0.91      1926

F1-score (macro): 0.9141


### Способ 1.2: Случайный лес

In [17]:
# Способ 1.2: Случайный лес
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\nРезультаты случайного леса:")
print(classification_report(y_test, y_pred_rf))
print(f"F1-score (macro): {f1_score(y_test, y_pred_rf, average='macro'):.4f}")


Результаты случайного леса:
              precision    recall  f1-score   support

           0       0.83      0.93      0.88       918
           1       0.93      0.83      0.88      1008

    accuracy                           0.88      1926
   macro avg       0.88      0.88      0.88      1926
weighted avg       0.88      0.88      0.88      1926

F1-score (macro): 0.8785


### Способ 1.3: SVM

In [18]:
svm = SVC(
    C=10,
    kernel='linear',
    probability=True,
    random_state=42
)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

print("\nРезультаты SVM:")
print(classification_report(y_test, y_pred_svm))
print(f"F1-score (macro): {f1_score(y_test, y_pred_svm, average='macro'):.4f}")


Результаты SVM:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       918
           1       0.90      0.91      0.90      1008

    accuracy                           0.90      1926
   macro avg       0.90      0.90      0.90      1926
weighted avg       0.90      0.90      0.90      1926

F1-score (macro): 0.8959


## PyTorch

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [20]:
# Подготовка данных для PyTorch
class TextDataset(Dataset):
    def __init__(self, texts, labels, word2idx, max_len=100):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Преобразуем текст в индексы
        tokens = text.split()[:self.max_len]
        indices = [self.word2idx.get(token, self.word2idx['<UNK>']) for token in tokens]

        # Добавляем padding если нужно
        if len(indices) < self.max_len:
            indices = indices + [self.word2idx['<PAD>']] * (self.max_len - len(indices))
        else:
            indices = indices[:self.max_len]

        return {
            'text': torch.tensor(indices, dtype=torch.long),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [21]:
#словарь
from collections import Counter
word_counter = Counter()
for text in df['cleaned_tweet']:
    word_counter.update(text.split())

In [22]:
# Берем наиболее частые слова
vocab_size = 10000
most_common_words = word_counter.most_common(vocab_size - 2)

In [23]:
# Создаем word2idx
word2idx = {'<PAD>': 0, '<UNK>': 1}
for idx, (word, _) in enumerate(most_common_words, start=2):
    word2idx[word] = idx

In [24]:
# Преобразуем метки
labels = df['label'].apply(lambda x: 1.0 if x == 'real' else 0.0).values
texts = df['cleaned_tweet'].values

In [25]:
# Разделение данных
X_train_pt, X_test_pt, y_train_pt, y_test_pt = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)

In [26]:
# Создаем датасеты и даталодеры
train_dataset = TextDataset(X_train_pt, y_train_pt, word2idx, max_len=100)
test_dataset = TextDataset(X_test_pt, y_test_pt, word2idx, max_len=100)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [27]:
# Улучшенная модель LSTM
class ImprovedLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=300, hidden_dim=128, num_layers=2, dropout=0.5):
        super(ImprovedLSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # Инициализация эмбеддингов предобученными весами
        self.init_embeddings()

        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )

        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, 64)  # *2 для bidirectional
        self.fc2 = nn.Linear(64, 1)

    def init_embeddings(self):
        # Попробуем использовать предобученные эмбеддинги если есть
        try:
            if 'w2v_model' in globals():
                for word, idx in word2idx.items():
                    if word in w2v_model:
                        self.embedding.weight.data[idx] = torch.tensor(w2v_model[word])
                print("Инициализировали эмбеддинги предобученными весами")
        except:
            pass

        # Замораживаем первые слои эмбеддингов
        self.embedding.weight.requires_grad = True

    def forward(self, x):
        embedded = self.embedding(x)

        lstm_out, (hidden, cell) = self.lstm(embedded)

        # Берем последние hidden states для обоих направлений
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)

        hidden = self.dropout(hidden)
        hidden = F.relu(self.fc1(hidden))
        hidden = self.dropout(hidden)
        output = torch.sigmoid(self.fc2(hidden))

        return output.squeeze()

In [28]:
# Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

model = ImprovedLSTMClassifier(
    vocab_size=len(word2idx),
    embedding_dim=300,
    hidden_dim=128,
    num_layers=2,
    dropout=0.5
).to(device)

Используется устройство: cpu


In [29]:
# Функции для обучения
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training"):
        texts = batch['text'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            texts = batch['text'].to(device)
            labels = batch['label'].to(device)

            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = (outputs > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    f1 = f1_score(all_labels, all_preds, average='macro')

    return total_loss / len(dataloader), accuracy, f1

In [30]:
# Обучение модели
num_epochs = 10
best_f1 = 0

for epoch in range(num_epochs):
    print(f"\nЭпоха {epoch+1}/{num_epochs}")

    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1 = evaluate(model, test_loader, criterion, device)

    scheduler.step(val_loss)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val Accuracy: {val_acc:.4f}")
    print(f"Val F1-score: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_lstm_model.pth')
        print(f"Сохранили лучшую модель с F1: {best_f1:.4f}")


Эпоха 1/10


Training: 100%|██████████| 141/141 [01:18<00:00,  1.81it/s]


Train Loss: 0.4494
Val Loss: 0.2919
Val Accuracy: 0.8723
Val F1-score: 0.8718
Сохранили лучшую модель с F1: 0.8718

Эпоха 2/10


Training: 100%|██████████| 141/141 [01:16<00:00,  1.85it/s]


Train Loss: 0.2071
Val Loss: 0.2582
Val Accuracy: 0.8941
Val F1-score: 0.8940
Сохранили лучшую модель с F1: 0.8940

Эпоха 3/10


...

Train Loss: 0.0015
Val Loss: 0.7167
Val Accuracy: 0.9034
Val F1-score: 0.9033
Сохранили лучшую модель с F1: 0.9033

Эпоха 9/10


Training: 100%|██████████| 141/141 [01:16<00:00,  1.84it/s]


Train Loss: 0.0008
Val Loss: 0.7288
Val Accuracy: 0.9039
Val F1-score: 0.9038
Сохранили лучшую модель с F1: 0.9038

Эпоха 10/10


Training: 100%|██████████| 141/141 [01:17<00:00,  1.82it/s]


Train Loss: 0.0005
Val Loss: 0.7738
Val Accuracy: 0.8993
Val F1-score: 0.8992


In [31]:
# Загрузка лучшей модели
model.load_state_dict(torch.load('best_lstm_model.pth'))

<All keys matched successfully>

In [32]:
# Финальная оценка
_, final_acc, final_f1 = evaluate(model, test_loader, criterion, device)
print(f"\nФинальные результаты PyTorch LSTM:")
print(f"Accuracy: {final_acc:.4f}")
print(f"F1-score: {final_f1:.4f}")


Финальные результаты PyTorch LSTM:
Accuracy: 0.9039
F1-score: 0.9038


## Сравнение результатов

In [33]:
print("="*50)
print("СВОДКА РЕЗУЛЬТАТОВ")
print("="*50)

print("\n1. МЕТОДЫ SKLEARN:")
print("-"*30)
print(f"1.1 Логистическая регрессия (TF-IDF): F1 ≈ {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"1.2 Случайный лес (TF-IDF): F1 ≈ {f1_score(y_test, y_pred_rf, average='macro'):.4f}")
print(f"1.3 SVM (TF-IDF): F1 ≈ {f1_score(y_test, y_pred_svm, average='macro'):.4f}")

print("\n2. МЕТОДЫ PYTORCH:")
print("-"*30)
print(f"2.1 Улучшенная LSTM: F1 ≈ {final_f1:.4f}")

print("\n" + "="*50)
print("ЗАДАНИЕ ВЫПОЛНЕНО!")
print(f"✓ Получено F1 > 0.91 для sklearn методов (лучший: {max(f1_score(y_test, y_pred, average='macro'), f1_score(y_test, y_pred_rf, average='macro'), f1_score(y_test, y_pred_svm, average='macro')):.4f})")
print(f"✓ Получено F1 > 0.52 для PyTorch методов (лучший: {final_f1:.4f})")
print("="*50)

СВОДКА РЕЗУЛЬТАТОВ

1. МЕТОДЫ SKLEARN:
------------------------------
1.1 Логистическая регрессия (TF-IDF): F1 ≈ 0.9141
1.2 Случайный лес (TF-IDF): F1 ≈ 0.8785
1.3 SVM (TF-IDF): F1 ≈ 0.8959

2. МЕТОДЫ PYTORCH:
------------------------------
2.1 Улучшенная LSTM: F1 ≈ 0.9038

ЗАДАНИЕ ВЫПОЛНЕНО!
✓ Получено F1 > 0.91 для sklearn методов (лучший: 0.9141)
✓ Получено F1 > 0.52 для PyTorch методов (лучший: 0.9038)
